In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
from spyne.application import Application
from spyne.decorator import srpc
from spyne.service import ServiceBase
from spyne.model.primitive import Integer
from spyne.model.primitive import Unicode
from spyne.model.complex import Iterable
from spyne.protocol.http import HttpRpc
from spyne.protocol.json import JsonDocument
from spyne.server.wsgi import WsgiApplication
import hashlib
import redis

class DiminishedTriadService(ServiceBase):
    global NUMBACKENDS
    global NUMBACKUPS
    NUMBACKENDS = 100 # this determines what we are modding by
    NUMBACKUPS = 1
    
    def getRingPos(key):
        return int(hashlib.sha1(key).hexdigest(), 16) % NUMBACKENDS
    
    hash_to_back_map = {
                            getRingPos("127.0.0.1:30001"):["127.0.0.1","30001"],
                            getRingPos("127.0.0.1:30002"):["127.0.0.1","30002"],
                            getRingPos("127.0.0.1:30003"):["127.0.0.1","30003"]
                        }
    
    def getNextNBackends(key):
        keys = hash_to_back_map.keys()
        keys = append(keys,getRingPos(key))
        keys = sorted(keys)
        index = keys.index(getRingPos(key))
        out = []
        for i in range(NUMBACKUPS):
            if keys[(index + i)%len(keys)] in hash_to_back_map and hash_to_back_map(keys[(index + i)%len(keys)]) not in out:
                out = append(out,hash_to_back_map(keys[(index + i)%len(keys)]))
        print "getNextNBackends for key : ",key," returning : {",out,"}"
        return out
    

    def __init__():
        print "I am being instantiated !!!!"
        
    @srpc(Unicode, _returns=Unicode)
    def get(key):
        backs = getNextNBackends(key)
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        r = redis.StrictRedis(host=backs[0][0], port=backs[0][1], db=0)
        return r.get(key)
    
    @srpc(Unicode,Unicode, _returns=Unicode)
    def set(key,value):
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        backs = getNextNBackends(key)
        r = redis.StrictRedis(host=backs[0][0], port=backs[0][1], db=0)     
        return r.set(key, value)
    
if __name__ == '__main__':
    from wsgiref.simple_server import make_server
    logging.basicConfig(level=logging.DEBUG)
    logging.getLogger('spyne.protocol.xml').setLevel(logging.DEBUG)
    application = Application([DiminishedTriadService],
        tns='DiminishedTriadService',
        in_protocol=HttpRpc(validator='soft'),
        out_protocol=JsonDocument()
    )
    wsgi_app = WsgiApplication(application)
    server = make_server('127.0.0.1', 8000, wsgi_app)
    server.serve_forever()

ERROR:spyne.application:global name 'self' is not defined
Traceback (most recent call last):
  File "/Users/srao/anaconda/lib/python2.7/site-packages/spyne-2.11.0-py2.7.egg/spyne/application.py", line 144, in process_request
    ctx.out_object = self.call_wrapper(ctx)
  File "/Users/srao/anaconda/lib/python2.7/site-packages/spyne-2.11.0-py2.7.egg/spyne/application.py", line 199, in call_wrapper
    retval = ctx.descriptor.service_class.call_wrapper(ctx)
  File "/Users/srao/anaconda/lib/python2.7/site-packages/spyne-2.11.0-py2.7.egg/spyne/service.py", line 207, in call_wrapper
    return ctx.function(*ctx.in_object)
  File "<ipython-input-1-5c783e712aed>", line 56, in set
    backs = self.getNextNBackends(key)
NameError: global name 'self' is not defined
